# Introduction

In this notebook, we will discuss a technique to analyze the available literature with the goal of extracting relevant information surrounding COVID-19 risk factors. The notebook is divided into two parts: 1) a walkthrough of the technique which we use to initially extract relevant information and 2) a cross-referencing with other datasets in order to further our findings. Both parts are described at a high level below. We delve into much greater detail during the actual analysis.

In Part 1, we begin by reading in the majority of the data available in the official Kaggle dataset. We will organize and clean the data, focusing on the titles and body text of the scholarly articles and research papers we have. We will then run a Latent Dirichlet Allocation, or LDA (source: https://stackabuse.com/python-for-nlp-topic-modeling/), to determine the various topics discussed in the data. Our focus will be to study a specific aspect of COVID-19 risk factors: public health mitigation measures that could be effective for control of the virus. We will filter the data based on the LDA to include papers which discuss the latter. Finally, we will run a sentiment analysis on the remaining data to extract the top ten most positive and top ten most negative papers, before combing through them manually to determine the ones relevant to our topic. Note that this technique is generalizable to any topic of our choice, which is what makes it powerful.

In Part 2, we will gather other datasets related to COVID-19. The datasets we use will focus particularly on the impact the virus has had on different countries and how those countries have responded. We aim to visualize and cross-reference this data with the findings from Part 1, in order to reach a more refined conclusion regarding public health measures that may be useful in containing the spread of COVID-19.

# Part 1: Latent Dirichlet Allocation and Sentiment Analysis

We begin by reading in the official Kaggle dataset and organizing the data into a table. Below, we import the necessary modules and libraries.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# We have added our own imports as well

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # helpful in reading in data
%matplotlib inline
import matplotlib.pyplot as plt #for visualizations
import seaborn as sns 

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

import glob

# Any results you write to the current directory are saved as output.

We read in the metadata and data below. For the rather dull details of how the data is read in, please refer to the following link: https://www.tutorialspoint.com/How-to-use-Glob-function-to-find-files-recursively-in-Python

In [ ]:
import os
print(os.listdir("../input"))

In [ ]:
metadata = pd.read_csv('../input/CORD-19-research-challenge/metadata.csv')

path = '/kaggle/input/CORD-19-research-challenge'

files = [f for f in glob.glob(path + "**/**/*.json", recursive=True)]

"""The text data for each paper is separated into multiple sections, and so we combine
it with this function for ease of analysis."""
def combine_text(text_dict):
    text_lst = [lst['section'] + ": " + lst['text'] if lst['section'] else "Unnamed: " + lst['text'] for lst in text_dict]
    return "\n".join(text_lst)   
    
data_dict = {'title' : [], 'paper_id' : [], 'text' : []} # We exclude abstract for now

for file in files:
    with open(file) as curr_json:
        curr_data = json.load(curr_json)
        # We want paper_id, metadata, and body_text as columns. We extract the title from metadata.
        # The keys body_text and abstract have values which are lists of dictionaries, with each dictionary containing the following keys:
        # text, cite_spans, ref_spans, and section. We will combine these into a single string, with each text entry separated
        # by a newline and prefaced by its section (if the section name exists). For now, we ignore cite_spans and ref_spans.
        data_dict['title'].append(curr_data['metadata']['title'])
        data_dict['paper_id'].append(curr_data['paper_id'])
        # data_dict['abstract'].append(combine_text(curr_data['abstract']))
        data_dict['text'].append(combine_text(curr_data['body_text']))

cord_data = pd.DataFrame(data_dict)
cord_data

Below, we create a vocabulary of all the words in the titles of the papers in our dataset, which we will need to run LDA. We remove all words that either 1) appear in more than 80% of the titles or 2) appear in fewer than 2 of the titles, and we remove all stop words.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(cord_data['title'].values.astype('U'))
doc_term_matrix

Each of the 59311* paper titles is represented as a column vector with 19886 entries, which implies that our dictionary has 19886 entries. Below, we will run the actual LDA. Note that the parameter "n_components" specifies how many topics we want our data divided into. Let us start with 10, and see if we find anything interesting. The code will create topics, as well as assign a probability per topic for each of the words in our vocabulary. Note that LDA will not infer topics on its own. It will simply assign topics based on the probability that each word in our dictionary appears in each respective topic. We must determine the topics ourselves by examining the highest-probability words in each category.

*Note this number may be inaccurate, as the dataset is being continually updated. It represents the number seen at the time this notebook was made public.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=10, random_state=42)
LDA.fit(doc_term_matrix)

In [ ]:
# We print out LDA.components_, a 2-D array containing the probability values of each word in our dictionary for each topic
# As one can see, it is not very informative in the current format
LDA.components_

Below, we print out the top twenty words for each of the topics we created. The code works by sorting the words in each topic by probability and accessing them by their indices.

In [ ]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 20 words for topic #{i+1}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-1:-21:-1]])
    print('\n')


Scanning through the topics above, we can begin to form an idea of the different topics that the literature discusses. For example, Topic #1 appears to discuss literature pertaining to the development of a vaccine, whereas Topic #8 seems related to the biological aspects of the virus. Recall that in this notebook, we are interested in public mitigation measures that may be effective for control of the virus. At a glance, Topic #4 looks promising. We will proceed by filtering our data to only include papers whose titles include the top ten words in the ninth topic. But first, we visualize the top words in Topic #4 as a word cloud. It is much easier to see the relation to public health visually.

In [ ]:
choice_topic = LDA.components_[3]
feature_names = count_vect.get_feature_names()    
top_150_topic = [feature_names[i] for i in choice_topic.argsort()[-1:-151:-1]]

# Citation: https://www.datacamp.com/community/tutorials/wordcloud-python
from wordcloud import WordCloud
top_150_topic_indices = choice_topic.argsort()[-1:-151:-1] # The indices of the top ten words
freqs = [int(choice_topic[i]) for i in top_150_topic_indices] # estimate of frequencies of the words
double_lst = [[top_150_topic[i]] * int(freqs[i] / 150) for i in range(150)]
multitext = " ".join([item for lst in double_lst for item in lst]) # Combine text into the string format needed for WordCloud
# Create and generate a word cloud image:
wordcloud = WordCloud(background_color=None, mode='RGBA').generate(multitext)
# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
top_ten = [feature_names[i] for i in choice_topic.argsort()[-1:-11:-1]]
public_health = cord_data[cord_data['title'].str.contains('|'.join(top_ten))] # This regex checks for any of the top ten words
public_health

Let us now run a sentiment analysis and see what we can discover. We will use the VADER Lexicon dataset in order to do this. The technique we use is largely based on an assignment (Homework 4) from UC Berkeley's second course in data science, DATA 100: Principles and Techniques of Data Science.

Before continuing, one might ask what our purpose is in doing this sentiment analysis. While this technique is generalizable to any of the topics discovered through the LDA we ran above, our focus here lies in studying public health mitigation measures that could be effective for control of coronavirus. There are far too many papers and articles in our table above to comb through by hand, even after filtering based on the LDA. The idea is that by running a sentiment analysis, we can extract papers containing titles with strong sentiments (either positive or negative). While this is certainly not comprehensive, the hope is that some of these high-sentiment papers contain information which could prove useful to the medical community.

We realize that the reasoning behind performing a sentiment analysis here may be confusing. Essentially, we require a way to filter the data further, and select papers from the corpus relevant to our topic. Sentiment analysis acts as a pseudo-random filter, but with the added effect of isolating papers with strong sentiments, which are likely to contain useful information. We are aware that there are better techniques out there, and as such this is one aspect of the analysis which we hope to improve in future research.

In [ ]:
lexicon = pd.read_csv('../input/vader-lexicon/vader_lexicon/vader_lexicon.txt', sep='\t', names = ['Words', 'polarity', 'drop1', 'drop2'], header=None)
lexicon = lexicon.drop(columns=['drop1', 'drop2']).set_index('Words').rename_axis(None)
lexicon.head()

The lexicon is in lowercase, and so we will convert the titles in our public health set to be in lowercase and remove punctuation from the titles, creating a new column called "raw title." This will lead to a more accurate sentiment analysis.

In [ ]:
punctuation_regex = r'[^\w\s]' # Searches the dataset for anything not a word character or a space character
public_health['raw title'] = public_health['title'].str.replace(punctuation_regex, ' ').str.lower()
public_health

Below, we transform our data into a form that is easier to analyze. Each title has a row for every word in it, storing what the word is and what its position is in the title. The code that creates it is, in my opinion (and UC Berkeley course staff's opinion), some very cool Pandas hacking.

In [ ]:
tidy_format = pd.DataFrame(public_health.loc[:, ['raw title']]['raw title'].str.split(expand=True).stack()).reset_index(level=1)
tidy_format = tidy_format.rename_axis(None).rename(columns={'level_1':'num', 0:'word'})
tidy_format

Now, we will add a polarity column to the public_health dataframe. We take each word for each title, and we sum the polarities together. We assign a total polarity value for each title. Notice that after calculating the sums, we standardize the values. Otherwise, the results would be biased, as longer titles would have higher absolute polarity values, since they have more words. 

In [ ]:
public_health = public_health.sort_index()
with_pols = tidy_format.merge(lexicon, how='left', left_on='word', right_index=True).sort_index() # Merge tidy table with lexicon to match words with polarity
# The index below identifies each unique article/paper
# Thus, grouping by it gives us a total polarity value for the title of each article
# Notice we divide by the count to normalize values
polarities = with_pols.reset_index().groupby('index').agg('sum').sort_index()['polarity']
polarities = (polarities - np.mean(polarities)) / np.std(polarities)
public_health['polarity'] = polarities
public_health

Below, we print the ten most positive titles and the ten most negative titles.

In [ ]:
print('Most positive titles:')
for t in public_health.sort_values('polarity', ascending=False).head(20)['title']:
    print('\n  ', t)

In [ ]:
print('Most negative titles:')
for t in public_health.sort_values('polarity').head(20)['title']:
    print('\n  ', t)

We are interested in answering the question of what public health mitigation measures might be effective for control of the virus. Based on what we see printed out above, the following papers (copied as given, so there may be grammatical errors) might prove useful: 

* "In the era of corona virus: health care professionals’ knowledge, attitudes, and practice of hand hygiene in Saudi primary care centers: a cross-sectional study"
* "Are health care workers protected? An observational study of selection and removal of personal protective equipment in Canadian acute care hospitals"
* "Cough aerosol in healthy participants: fundamental knowledge to optimize droplet-spread infectious respiratory disease management Cough aerosol in healthy participants: fundamental knowledge to optimize droplet-spread infectious respiratory disease management"
* "Management of a Large Middle East Respiratory Syndrome Outbreak in a Tertiary Care Hospital: a Qualitative Case Study Management of a Large Middle East Respiratory Syndrome Outbreak Management of a Large Middle East Respiratory Syndrome Outbreak in a Tertiary Care Hospital ... "
* "Journal Pre-proof The effective use of telemedicine to save lives and maintain structure in a healthcare system: Current response to COVID-1 The Effective Use of Telemedicine to Save Lives and Maintain Structure in a Healthcare System: Current Response to COVID-19"


Below, we take a look at the text for one of these articles as a sample of what kind of information might prove useful to the medical community during this time.

In [ ]:
text1 = public_health[public_health['title'] == 'Management of a Large Middle East Respiratory Syndrome Outbreak in a Tertiary Care Hospital: a Qualitative Case Study Management of a Large Middle East Respiratory Syndrome Outbreak Management of a Large Middle East Respiratory Syndrome Outbreak in a Tertiary Care Hospital: a Qualitative Case Study Perceptions of Management of a Large Middle East Respiratory Syndrome Outbreak in a Tertiary Care Hospital: a Qualitative Study Management of a Large Middle East Respiratory Syndrome Outbreak Perceptions of Post-break out Management by management and healthcare workers of a Middle East Respiratory Syndrome Outbreak in a Tertiary Care Hospital: a Qualitative Study Management of a Large Middle East Respiratory Syndrome Outbreak']
print(text1['text'].values[0])

We present some highlights from the above paper for the reader's viewing pleasure below. At the very least, skimming the bullet points is recommended, as we will return to them later in the notebook. We include a brief summary at the end for those pressed for time.

* "This work can serve as a tool to guide organizational actions in the control of rare infectious outbreaks in an advanced health care setting."

* "**Stress of the outbreak**:: Across all participants, data indicated that the MERS outbreak was perceived to constitute an episode of the most pressing and serious significance to the organization. Variously, it was described as a" serious and straining situation" and a "situation that carries national and international significance". Frontline HCWs and senior decision-makers alike not only perceived this high gravity, but also the high demands the outbreak placed on them personally."

* "**Factors perceived to have contributed to outbreak occurrence**:: Strong consensus existed among participants that the organization's "growing reliance on ED boarding" was a main contributor to the outbreak. While some lamented that this build up was permitted by senior decision-makers to occur over the long-term, others saw the contributing factors as being more upstream. These factors are related to the high trust patients had for the NGHA, the relative lack of primary care services and problems associated with patient flows at the level of the city, communication and teamwork between the city hospitals, and the lack of a national plan that allows hospitals across different sectors to share the burden of the growing patient population. Another leading factor to the outbreak was the "inadequate implementation of IPC guidelines" within the extremely busy ED and the suboptimal recognition of the importance of IPC principles across the organization. empowering HCWs to work collaboratively to address the outbreak, involving staff in outbreak control and stimulating staff resilience and teamwork. The culture was one of "collective leadership practice" (table 1) with leaders being seen to have practiced high levels of availability, visibility, empowerment of middle management and strong links with frontline staff."

* "**Factors inhibiting outbreak control**:: Factors that were perceived to challenge the outbreak management were mostly related to historical management practices and context. The organization was perceived to be acutely and chronically slow in responding to the outbreak. The main factors perceived to limit the capacity of handling the outbreak adequately were related to: Factors inhibiting outbreak control:: 1. Initial logistic barriers to promptly implement the escalation process outlined in the institution's Infectious Disease Epidemic Plan (IDEP) ( Anxiety increased among health professionals, due to the "lack of appropriate staff counseling and mental health support". The psychological demands on frontline health professionals escalated, not only by undertaking tasks associated with the IDEP implementation, but also due to a range of coalescing factors; most notably the need to manage the outbreak simultaneously with the closure of most of the institution's units, combined with the negative media commentary. Almost all frontline participants have repeatedly expressed the need for a center that provides counseling and mental health support to employees in such stressful situations."

* "**Long-term institutional gains in response to the outbreak management**:: 1-Improvement in the facility to be more compatible with IC guidelines. KAMC-R to work collaboratively to address long-standing issues around IPC, ED boarding and nursing standards."

                                   **BRIEF SUMMARY**

For context, this paper pertains to MERS-CoV, first identified in Saudi Arabia in 2012. The study focuses on best (and worst) practices in health care settings with regard to controlling an outbreak. The paper dicusses that the outbreak initially occurred because of a lack of national planning which made it difficult for hospitals throughout the nation to handle the growing number of patients. The outbreak was then difficult to control due to a slow response combined with lack of mental support and counseling for health professionals. However, the silver lining gained from this was the improvement of institutions in the long run with regard to being prepared for such an outbreak. We shall later see that this made a large difference in why South Korea had such an effective response to the COVID-19 outbreak.

# Part 2: Gathering Relevant Info from other Datasets

Below, we read in a dataset which contains cumulative data on confirmed cases, recoveries, and deaths related to COVID-19 by country. We initially plot growth curves for different countries and provinces to get an idea of how the virus has spread in different places. Afterward, we delve deeper with two case studies: New York vs. California, and Italy vs. South Korea.

In [ ]:
virus_cases = pd.read_csv('../input/corona-virus-report/covid_19_clean_complete.csv')
virus_cases[virus_cases['Country/Region'].str.contains('hina')]

In [ ]:
# Note that all values are cumulative
# It should be noted that values below are estimates, because it is possible that there was not an update every day in each province
# Our goal is simply to get an idea of the curve's general structure
china = virus_cases[virus_cases['Country/Region'] == 'China']
china.loc[:, 'Date'] = pd.to_datetime(china['Date']) # Will allow for ease of sorting later on, should we need it
china.loc[:, 'Date'] = china['Date'].dt.date
china = china.groupby('Date')[['Confirmed', 'Deaths', 'Recovered']].sum() # Aggregate values from different provinces together
sns.lineplot(x=np.arange(china.shape[0]), y=china['Confirmed'], label = 'Confirmed')
sns.lineplot(x=np.arange(china.shape[0]), y=china['Recovered'], label = 'Recovered')
sns.lineplot(x=np.arange(china.shape[0]), y=china['Deaths'], label = 'Deaths')
plt.xlabel('Days since Jan 21st')
plt.ylabel('Count')
plt.title('Confirmed Cases, Recoveries, and Deaths for China')
plt.legend()

In [ ]:
# Clean up format of data
virus_cases.loc[:, 'Date'] = pd.to_datetime(virus_cases['Date'])
virus_cases['Province/State'] = virus_cases.fillna('No specified province/state')
virus_cases = virus_cases.sort_values('Date', ascending=False)
by_country = virus_cases.groupby(['Country/Region', 'Province/State']).first() # Captures necessary info because values cumulative
by_country.reset_index(inplace=True)
by_country = by_country.groupby('Country/Region')[['Confirmed', 'Deaths', 'Recovered']].sum()
by_country.head(40)

Below, we visualize the confirmed cases for the top ten most affected countries as a bar graph. What we see below is an estimate due to the way we aggregated our data above, but is roughly consistent with the most up-to-date data found at https://www.worldometers.info/coronavirus/#countries. Therefore, we can expect that our visualization is reasonable accurate.

In [ ]:
top_ten_countries = by_country.sort_values('Confirmed', ascending=False).head(10)
top_ten_countries

In [ ]:
sns.barplot(x=top_ten_countries.index, y=top_ten_countries['Confirmed'])
plt.title('Confirmed Cases for Top Ten Most Affected Countries')

Below, we read in a dataset that focuses specifically on the United States. We can see at a glance that it contains a lot of useful information about the number of cases per state, as well as the testing rate (which is important when it comes to comparing the curves of different states.

In [ ]:
united_states = pd.read_csv('../input/covid19-in-usa/us_states_covid19_daily.csv')
united_states.head()

The above dataset will prove useful for further analysis, but initially we simply want to visualize the growth curve in various states in the U.S. In order to do so, we will need a cumulative dataset that includes state data. We read one in below. The reason we did not read this one in initially is that the data for some dates (March 8th in particular) appears to be faulty due to data entry errors. We simply remove these dates in our visualizations. We start by visualizing the growth curve of confirmed cases, recoveries, and deaths for the entire United States.

In [ ]:
from datetime import datetime

cases = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')
us = cases[cases['Country/Region'] == 'US']
us.loc[:, 'Last Update'] = pd.to_datetime(us['Last Update'])
us.loc[:, 'Last Update'] = us['Last Update'].dt.date
us = us.groupby('Last Update')[['Confirmed', 'Deaths', 'Recovered']].sum()
#This is a bad way to group, because days that do not have an update will be inaccurate
#For now, we will drop the main outliers--March 8 and March 13
us = us.reset_index()
us = us[(us['Last Update'] != datetime.strptime('2020-03-08', '%Y-%m-%d').date())
       & (us['Last Update'] != datetime.strptime('2020-03-13', '%Y-%m-%d').date())]
sns.lineplot(x=np.arange(us.shape[0]), y=us['Confirmed'], label = 'Confirmed')
sns.lineplot(x=np.arange(us.shape[0]), y=us['Recovered'], label = 'Recovered')
sns.lineplot(x=np.arange(us.shape[0]), y=us['Deaths'], label = 'Deaths')
plt.xlabel('Days since first observation')
plt.ylabel('Count')
plt.title('Confirmed Cases, Recoveries, and Deaths for United States')
plt.legend()

Below, we write code to isolate the data for a single state from the dataframe and to visualize the growth curve for a given state. We then proceed to visualize the curves for New York, California, and Washington on the same plot and different plots. Notice that while the curves look similar when plotted separately, when plotted together New York's growth rate is clearly much faster.

In [ ]:
us_cases = cases[cases['Country/Region'] == 'US']
us_cases['Last Update'] = pd.to_datetime(us_cases['Last Update'])

# Filter dataframe to include a single state
def get_state_df(state):
    state_df = us_cases[us_cases['Province/State'] == state]
    state_df.loc[:, 'Last Update'] = state_df['Last Update'].dt.date
    state_df = state_df.sort_values('Last Update', ascending = False)
    state_df.groupby('Last Update')[['Confirmed', 'Deaths', 'Recovered']].first() # Takes the latest reading from each day
    state_df.reset_index(inplace=True)
    state_df = state_df.sort_values('Last Update', ascending = True) # Puts data back in chronological order
    # The data has March 22nd mislabeled as March 8th. It leads to a misrepresentative visualization, so we drop it
    state_df = state_df[state_df['Last Update'] != datetime.strptime('2020-03-08', '%Y-%m-%d').date()]
    return state_df

#Draw growth curve for state
def visualize_state(state):
    state_df = get_state_df(state)
    sns.lineplot(x=np.arange(state_df.shape[0]), y=state_df['Confirmed'], label = 'Confirmed')
    sns.lineplot(x=np.arange(state_df.shape[0]), y=state_df['Recovered'], label = 'Recovered')
    sns.lineplot(x=np.arange(state_df.shape[0]), y=state_df['Deaths'], label = 'Deaths')
    plt.xlabel('Days since 1st Measurement for ' + state)
    plt.ylabel('Count')
    plt.title('Growth Curve for ' + state)
    plt.legend()

cali = get_state_df('California')
ny = get_state_df('New York')
washington = get_state_df('Washington')
sns.lineplot(x=np.arange(cali.shape[0]), y=cali['Confirmed'], label = 'California')
sns.lineplot(x=np.arange(ny.shape[0]), y=ny['Confirmed'], label = 'New York')
sns.lineplot(x=np.arange(washington.shape[0]), y=washington['Confirmed'], label = 'Washington')
plt.xlabel('Days since 1st observation')
plt.ylabel('Count')
plt.title('Growth Curve for New York, Washington, and California')
plt.legend()

In [ ]:
visualize_state('California')

In [ ]:
visualize_state('New York')

In [ ]:
visualize_state('Washington')

Below, we read in and organize another dataset--one that contains information about the different measures countries have implemented to mitigate the spread of the virus.

In [ ]:
measures = pd.read_csv('../input/covid19-containment-and-mitigation-measures/COVID 19 Containment measures data.csv')
measures

In [ ]:
country_measures = pd.DataFrame(measures.groupby('Country')['Keywords'].agg(lambda x: list(x.values)))
country_measures = country_measures.reset_index()
country_measures

Below, we create a dataframe that contains only measures for the countries with the top ten most confirmed cases (as well as the states of New York and California), to get an idea of the data we will be exploring in the case studies below.

In [ ]:
top_ten_country_measures = country_measures[(country_measures['Country'].isin(list(top_ten_countries.index))) | 
                (country_measures['Country'] == 'US:New York') | (country_measures['Country'] == 'US:California')]


In [ ]:
top_ten_country_measures

In [ ]:
top_ten_country_measures['Keywords'].values[9]

# Case Study: New York vs. California

Below, we will compare the states of New York and California in the United States. New York has significantly more confirmed cases than California (as a caveat, it also has a higher testing rate, which we will address later). We want to determine if the measures put in place by both states played a role in the growth rate of cumulative virus infections. We create two interactive line plots which plot the number of confirmed cases in each state through time. By hovering over a particular location on the line, we can see the measures put in place on that date. The visualization will help us to see if there exists any correlation between how quickly action was taken and the eventual spread of the virus.

Our code is explained in detail through comments; however, for ease of the reader, we provide a high-level overview of how the plot is created:
    
* We filter out the measures table from above to include only the measures implemented for the desired state.
* We sort and group by the date and aggregate all measures taken on the same day together into a list. We strip all other information from the table.
* Then, using our function defined above, we obtain a dataframe consisting of the number of cumulative confirmed cases by date for the desired state.
* We merge the two tables together on the date. Note that many dates with a large jump in confirmed cases did not have any specific measures implemented. To ensure we do not look these dates over, we perform a right merge.
* Using Plotly Express, we create a lineplot with the data from the merged table. We plot date on the x-axis, confirmed cases on the y-axis, and keywords of implemented measures on tooltips (which appear when one hovers over the line).

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode()
import plotly.express as px
init_notebook_mode(connected=True)

# This function creates a dataframe containing the number of confirmed cases along with the measures implemented for each date
def create_state_summary(state):
    # Isolate measures for state of choice, and extract date and keyword data
    state_measures = measures[measures['Country']=='US:{}'.format(state)][['Country', 'Date Start', 'Keywords']]
    
    state_measures['Date Start'] = pd.to_datetime(state_measures['Date Start']) # Will allow us to easily sort by the date
    state_measures = state_measures.sort_values('Date Start')
    
    # Reformat dataframe to aggregate all measures taken on the same date together into a list
    # Only one row for each date nowf
    state_measures = pd.DataFrame(state_measures.groupby('Date Start')['Keywords'].agg(lambda x: list(x.values)))
    
    # Get dataframe of cumulative confirmed cases for state
    
    state_confirmed = get_state_df(state)[['Last Update', 'Confirmed']]
    state_confirmed['Last Update'] = pd.to_datetime(state_confirmed['Last Update']) # Need to convert to matching type so that we can merge
    
    # Perform a right merge on the two dataframes
    # This enables us to keep the dates which do not have a corresponding measure implemented
    state_summary = state_measures.merge(state_confirmed, how='right', left_index=True, right_on='Last Update')
    state_summary = state_summary.fillna('N/A') # Ensures plot renders properly, just showing 'N/A' if a date has no new measure implemented
    return state_summary

# This function takes in the summary created above and created the desired plot.
def create_state_interactive_plot(state_name, state_summary):
    fig = px.line(state_summary, x="Last Update", y="Confirmed", title='{} Cases over Time'.format(state_name),
             hover_name="Last Update", hover_data=["Keywords"])
    fig.show()
    return

ny_summary = create_state_summary('New York')
create_state_interactive_plot('New York', ny_summary)

In [ ]:
cali_summary = create_state_summary('California')
create_state_interactive_plot('California', cali_summary)

Of course, the real value lies in comparison. Below, we plot California and New York's growth curve on the same plot.

In [ ]:
ny_summary = create_state_summary('New York')
cali_summary = create_state_summary('California')

ny_summary['State'] = np.array(['New York'] * ny_summary.shape[0])
cali_summary['State'] = np.array(['California'] * cali_summary.shape[0])

combined_summary = pd.concat([ny_summary, cali_summary], axis=0)
combined_summary

fig = px.line(combined_summary, x="Last Update", y="Confirmed", title='New York vs. California',
              hover_name="Last Update", hover_data=["Keywords"], color='State')

fig.show()

Both states have a curve that begins to grow at appoximately mid-March (a bit past March 15th, if we eyeball it). However, there is a key detail worth noting. If one hovers over both curves on March 12, the measure keywords are as follows:

* New York: school closure, indoor gatherings banned, cultural activity limitation, very large event cancellation or postponement
* California: outdoor gatherings banned, public hygiene, outdoor gatherings banned, outdoor gatherings banned, public annoncement, state of emergency

Ignoring the repetitiveness of the keywords, we can see the California took much more drastic action early on, banning all outdoor gatherings. By contrast, New York had only cancelled indoor gatherings at this point. Take particular note that on March 12, the number of confirmed cases in both states was nigh identical. California's quicker action continued. Though not visible on the plot, much of northern California issued a shelter-in-place the night of March 16th (see [https://www.cnn.com/2020/03/16/us/coronavirus-california-shelter-in-place/index.html](http://)), and the entire state followed suit three nights later on March 19th (see [https://www.wwlp.com/news/health/coronavirus/california-becomes-first-state-to-put-shelter-in-place-order-into-effect/](http://)). New York, on the other hand, did not have a lockdown take full effect until the night of March 22nd (see [https://nypost.com/2020/03/20/coronavirus-in-ny-cuomo-orders-lockdown-shuts-down-non-essential-businesses/](http://) -- note that the lockdown was announced on March 20th, but as per the article it went into full effect on Sunday, March 22nd at 8:00 PM EST).

The plot above is of course just observational data, so we cannot draw any definitive conclusions from it. That being said, it suggests a strong correlation between California's early actions and a slower spread rate of COVID-19. This lends support to the hypothesis that one of the most effective control measures of the virus is to order a total shelter-in-place, allowing people to go outside only for essential services. Even moreso, the data indicates that this shelter-in-place must be instituted *early on*, before the growth rate of the virus gets a chance to take off at all.

Of course, we must address one major concern regarding the observations above: what if the testing rates in both cities are vastly different, and in reality the growth curves are quite similar? To learn about the testing rates, we take another look at the informative dataframe from earlier in the notebook, isolating the relevant columns and rows from the most recent available data.

In [ ]:
tests_done = united_states.head(55) # We only want the most recent available data
tests_done = tests_done[(tests_done['state'] == 'CA') | (tests_done['state'] == 'NY')][['state','totalTestResults']]
# Below, we add in the populations of each state 
# Source: Google, which was sourced from the United States Census Bureau
tests_done['total'] = np.array([39510000, 19450000])
# Finally, we divide to get the testing rate
tests_done['testingRate'] = tests_done['totalTestResults'] / tests_done['total']
tests_done

Well. It would seem both rates are a bit abysmal. Even then, New York appears to have a testing rate of 2.5%, five times as much as California's rate of .5%. So, it is possible (and even likely) that the curves are somewhat closer to each other than they appear.

However, the data still suggests California has done a better job of containing COVID-19. If we do not consider proportions and look at only the raw values, New York has conducted a little more than two times as many tests as California. This should (and does) urge us to be cautious when viewing California's seemingly much flatter curve. That said, the difference in testing rate does not indicate a difference as drastic as the one we see in the graphs. Thus, we maintain our initial position that California's early actions likely made a difference in the spread of the virus.

# Case Study: Italy vs. South Korea

Below, we do the exact same thing that we did for California and New York (see the above section for a high-level overview of what the code does), but this time for Italy and South Korea. Notice we redefine the previous functions with very minor differences to now take in country names. Admittedly such repetitiveness is in poor taste from a computer science perspective, but we have chosen to include it for ease of use and documentation.

In [ ]:
cases['Last Update'] = pd.to_datetime(cases['Last Update'])
# Filter dataframe to include a single country
def get_country_df(country):
    country_df = cases[cases['Country/Region'] == country]
    country_df.loc[:, 'Last Update'] = country_df['Last Update'].dt.date
    country_df = country_df.sort_values('Last Update', ascending = False)
    country_df.groupby('Last Update')[['Confirmed', 'Deaths', 'Recovered']].first() # Takes the latest reading from each day
    country_df.reset_index(inplace=True)
    country_df = country_df.sort_values('Last Update', ascending = True) # Puts data back in chronological order
    # The data has March 22nd mislabeled as March 8th. It leads to a misrepresentative visualization, so we drop it
    country_df = country_df[country_df['Last Update'] != datetime.strptime('2020-03-08', '%Y-%m-%d').date()]
    return country_df

# This function creates a dataframe containing the number of confirmed cases along with the measures implemented for each date
def create_country_summary(country):
    # Isolate measures for country of choice, and extract date and keyword data
    country_measures = measures[measures['Country']==country][['Country', 'Date Start', 'Keywords']]
    
    country_measures['Date Start'] = pd.to_datetime(country_measures['Date Start']) # Will allow us to easily sort by the date
    country_measures = country_measures.sort_values('Date Start')
    
    # Reformat dataframe to aggregate all measures taken on the same date together into a list
    # Only one row for each date now
    country_measures = pd.DataFrame(country_measures.groupby('Date Start')['Keywords'].agg(lambda x: list(x.values)))
    
    # Get dataframe of cumulative confirmed cases for country
    
    country_confirmed = get_country_df(country).loc[:, ['Last Update', 'Confirmed']].drop_duplicates(keep='first')
    country_confirmed['Last Update'] = pd.to_datetime(country_confirmed['Last Update']) # Need to convert to matching type so that we can merge
    
    # Perform a right merge on the two dataframes
    # This enables us to keep the dates which do not have a corresponding measure implemented
    country_summary = country_measures.merge(country_confirmed, how='right', left_index=True, right_on='Last Update')
    country_summary = country_summary.fillna('N/A') # Ensures plot renders properly, just showing 'N/A' if a date has no new measure implemented
    return country_summary

# This function takes in the summary created above and created the desired plot.
def create_country_interactive_plot(country_name, country_summary):
    fig = px.line(country_summary, x="Last Update", y="Confirmed", title='{} Cases over Time'.format(country_name),
             hover_name="Last Update", hover_data=["Keywords"])
    fig.show()
    return

temp = create_country_summary('Italy')
create_country_interactive_plot('Italy', temp)

In [ ]:
temp = create_country_summary('South Korea')
create_country_interactive_plot('South Korea', temp)

In [ ]:
italy_summary = create_country_summary('Italy')
skorea_summary = create_country_summary('South Korea')

italy_summary['Country'] = np.array(['Italy'] * italy_summary.shape[0])
skorea_summary['Country'] = np.array(['South Korea'] * skorea_summary.shape[0])

combined_summary = pd.concat([italy_summary, skorea_summary], axis=0)
combined_summary

fig = px.line(combined_summary, x="Last Update", y="Confirmed", title='Italy vs. South Korea',
              hover_name="Last Update", hover_data=["Keywords"], color='Country')


fig.show()

The measure keywords are a bit hard to follow in this plot, especially because of the overlap and truncated form of the measures taken. However, one can still glean useful information. For Italy:

* The government starting addressing the situation minimally in late January.
* In late February/early March, university closures, expansion of testing facilities, and activity cancellations took place.
* On March 9, a national quarantine was instituted. This is not visible on the plot, but for more information see [https://www.npr.org/sections/goatsandsoda/2020/03/09/813791575/italy-expands-quarantine-measures-nationwide-to-stem-spread-of-coronavirus](http://).
* On March 11, a general nonessential business suspension was instituted.
* In mid-to-late March, mandatory self-isolation was implemented for Italians returning to the country.

And for South Korea:

* In late January/early February, the government was already working on expansion of testing facilities and heightened public hygeine measures.
* During late February, social distancing was encouraged, and outside activities (including schooling) began to become limited.
* Starting mid-March, South Korea also began establishing location-based tracing of its citizens.

The difference this time does not appear to be institution of a quarantine (as was the case with New York and California). In fact, the measures taken by Italy might at first appear more drastic. So what made the difference? The answer appears to lie in South Korea's quick national action and implementation of efficient nationwide testing (visible on the plot as well). Further details on South Korea's measures can be found at [www.foreignaffairs.com/articles/united-states/2020-04-10/south-korea-offers-lesson-best-practices](http://).

Like most countries, South Korea implemented social distancing and quarantines in the wake of the virus. However, their success in containing COVID-19 appears to have been strongly enhanced by widespread testing and geo-tracing of its citizens (the entire country has fewer cases than California alone).

Combining this with what we learned from New York and California, the data appears to suggest that while quick lockdowns early on do help to flatten the curve, this effect can be greatly enhanced by instituting widespread testing in the early stages of spread--something which California did not do.

# Tying it All Together

Before concluding this notebook, we briefly tie together some of the exploratory data analysis we have done above, and explain how our process can be generalized to study any relevant topic of choice. Recall that after we conducted our initial Latent Dirichlet Allocation and sentiment analysis, we isolated a particular paper which pertained to control of a MERS-CoV outbreak. From the paper, we learned that one of the key factors in controlling an outbreak relates to a nationally coordinated effort surrounding health care. We can see evidence for the paper's claim in South Korea's curve above. As we discussed, South Korea had one of the most coordinated national responses to the COVID-19 outbreak of any country (more details on its response can be found in the link above), and it appears to have made quite the difference. In fact, South Korea's preparedness was a direct result of the lessons it learned from MERS-CoV. For details, see [https://thebulletin.org/2020/03/south-korea-learned-its-successful-covid-19-strategy-from-a-previous-coronavirus-outbreak-mers/](http://)

At a high level, this illustrates the primary goal of the technique outlined in this notebook for obtaining useful information regarding COVID-19 risk factors from the enormous corpus of data available. The process is outlined below for anyone who wishes to use it (each step can be completed with only minor modifications of the code above):

* Identify the risk factor that you would like to study. In our case, we focused on public health mitigation measures that could be effective for control.
* Run Latent Dirichlet Allocation on the available corpus of data. Look at the topics created, and identify which one relates most closely to your chosen one.
* Filter out the titles of your articles based on the top *n* (choice of *n* is left to the reader) words in your topic.
* On the remaining paper titles, run a sentiment analysis, and view the top 20 most positive and most negative papers.
* Manually determine and read/skim the papers which are most relevant to your topic of study.
* Gather further data pertaining to your topic of study. In our case, this would be the data on confirmed cases of the virus, testing rates, and mitigation measures.
* Conduct further exploratory data analysis with the aim of drawing a connection between what information is gleaned from the papers and what the additional data show.

# Strengths and Limitations

The primary strengths of the approach outlined above center around its generality and effectiveness. While we chose to focus on public health mitigation measures, the topics created by LDA extend far beyond that. We reproduce the output of our original LDA below to illustrate this:

> Top 20 words for topic #1:
> ['vaccine', 'cov', 'sars', 'mers', 'vaccines', 'virus', 'protein', 'based', 'recombinant', 'activity', 'efficacy', 'novel', 'mice', 'immune', 'development', 'design', 'anti', 'dna', 'responses', 'antibody']
> 
> 
> Top 20 words for topic #2:
> ['diseases', 'infectious', 'emerging', 'viral', 'review', 'new', 'human', 'research', 'molecular', 'therapeutic', 'current', 'future', 'biology', 'disease', 'infections', 'applications', 'potential', 'drug', 'host', 'antiviral']
> 
> 
> Top 20 words for topic #3:
> ['detection', 'pcr', 'using', 'based', 'time', 'assay', 'virus', 'rapid', 'real', 'development', 'human', 'viruses', 'evaluation', 'identification', 'amplification', 'respiratory', 'fever', 'protein', 'rt', 'molecular']
> 
> 
> Top 20 words for topic #4:
> ['health', 'public', 'global', 'study', 'care', 'disease', 'ebola', 'control', 'research', 'diseases', 'surveillance', 'international', 'infectious', 'based', 'response', 'emerging', 'systems', 'challenges', 'virus', 'risk']
> 
> 
> Top 20 words for topic #5:
> ['influenza', 'pandemic', 'infection', 'h1n1', 'patients', 'avian', '2009', 'infections', 'virus', 'hong', 'risk', 'study', 'kong', 'china', 'healthcare', 'associated', 'viral', 'clinical', 'analysis', 'h5n1']
> 
> 
> Top 20 words for topic #6:
> ['respiratory', 'syndrome', 'acute', 'coronavirus', 'severe', 'infections', 'infection', 'children', 'east', 'middle', 'viral', 'patients', 'human', 'tract', 'associated', 'virus', 'study', 'sars', 'clinical', 'mers']
> 
> 
> Top 20 words for topic #7:
> ['coronavirus', 'feline', 'infection', 'viruses', 'bovine', 'coronaviruses', 'human', 'pneumonia', 'bats', 'acquired', 'community', 'calves', 'virus', 'disease', 'novel', 'canine', 'cats', 'characterization', 'molecular', 'bat']
> 
> 
> Top 20 words for topic #8:
> ['virus', 'protein', 'cells', 'porcine', 'cell', 'rna', 'infection', 'coronavirus', 'replication', 'expression', 'viral', 'human', 'analysis', 'gene', 'hepatitis', 'type', 'diarrhea', 'interferon', 'host', 'infectious']
> 
> 
> Top 20 words for topic #9:
> ['access', 'disease', 'public', 'nd', 'nc', 'nih', 'viral', 'multiple', 'immunity', 'author', 'article', 'chronic', 'manuscript', 'hhs', 'supplementary', 'immune', 'org', 'license', 'spectrum', 'licenses']
> 
> 
> Top 20 words for topic #10:
> ['19', 'covid', 'disease', 'china', 'outbreak', 'epidemic', 'coronavirus', 'transmission', 'analysis', '2019', 'journal', 'sars', 'data', 'novel', 'pre', 'infectious', 'based', 'control', 'model', 'review']


Topic #10 appears related to the transmission dynamics of the virus, Topic #6 appears related to co-infections, and Topic #7 appears related to the replication of the virus. All three of these are other risk factors mentioned in the CORD-19 challenge, and an analagous process to the one carried out in this notebook can feasibly done for each of them. Furthermore, once the data is filtered, the sentiment analysis is reasonably likely to isolate relevant papers. Honing in on papers with high sentiment (whether positive or negative) prioritizes those whose titles likely convey something of substance about the desired topic (in particular because we run the sentiment analysis *after* LDA). Finally, comparing the findings with further data from other sources allows one to draw intriguing connections among the different aspects of the chosen topic, ideally learning something along the way.

Of course, our approach has its weaknesses as well. The sentiment analysis we conduct works fairly well, but is not the ideal technique. There is a distinct possibility that some of the more "scientifically-titled" papers, so to speak, contain words not present in the VADER lexicon, and thus are assigned a sentiment of 0 despite being potentially useful (as 0 is neither low nor high in this setting, these papers are ignored). Beyond this, our process is also primarily qualitiative, and would benefit from an introduction of some probability. For example, our case study visualizations which plot measures taken in conjunction with confirmed cases do suggest a correlation between certain measures and a flattening of the curve, but it would be useful to somehow quantify the likelihood of a certain measure having this effect.

The weaknesses of our approach suggest potential future studies, which we discuss below.

# Conclusion and Future Research

Our primary goal in conducting this exploratory data analysis was to determine public health mitigation measures that may be effective for control of COVID-19. The data suggest two key findings: 1) early implementation of social distancing and lockdown measures appear to be correlated with a substantial slowdown in the spread rate of the virus, and 2) a coordinated national effort surrounding organization of a country's health care system (especially with regard to testing) can be instrumental in "flattening the curve," as is commonly said nowadays.

In future notebooks, the analysis done here can be enhanced upon in a number of ways. In terms of the data visualizations, it would be tremendously useful for clarity purposes to create a plot which lists not only measure keywords, but (succinctly) displays detailed measures taken by different countries and states. For the analysis itself, the clear next step is to replace the sentiment analysis conducted after LDA with something more encompassing of all the data, so we do not gloss over potentially useful information. Once that is done, we will have a more refined dataset. The final step would then be to quantify the effect that different public health mitigation measures have on the growth curve of COVID-19 with a probabilistic model.

Let us see what the future brings.
